In [1]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro C:\Users\Adrian\Documents\repo\TimeSeries

2022-06-30 21:43:27,433 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-30 21:43:28,089 - root - INFO - ** Kedro project timeseries
2022-06-30 21:43:28,090 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-06-30 21:43:28,384 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-06-30 21:43:28,459 - root - INFO - ** Kedro project timeseries
2022-06-30 21:43:28,460 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


## To Do:
- Add ability to leverage
  - Cost of leverage
  - Margin
- Benchmark 

## Libraries

In [77]:
import numpy as np
import pandas as pd
from importlib import reload
import portfolio
import metrics
reload(portfolio)
reload(metrics)

from portfolio import DataLoader, Portfolio
from metrics import Metrics


## Data

In [154]:
prices = catalog.load("fund_prices")
prices

2022-06-30 23:43:49,318 - kedro.io.data_catalog - INFO - Loading data from `fund_prices` (ParquetDataSet)...


,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,2020-12-31,324.200,92.850000,91.710,3256.93,132.690000,121.410000,50.250000
1,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000
2,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
3,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
4,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
...,...,...,...,...,...,...,...,...
5322,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
5323,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
5324,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
5325,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210


In [155]:
sp500 = catalog.load("sp500")

2022-06-30 23:43:49,630 - kedro.io.data_catalog - INFO - Loading data from `sp500` (CSVDataSet)...


## Backtesting Portfolio

In [156]:
prices = DataLoader().prepare_data(prices)

In [157]:
prices

,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,1999-11-01,10.250,1.050148,10.155,69.13,0.597773,13.235224,7.096464
1,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210
2,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
3,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
4,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
...,...,...,...,...,...,...,...,...
5322,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
5323,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
5324,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
5325,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000


In [158]:
external_data = {
    "codes": ["EBAY", "AAPL"],
    "amounts": [30,30],
    "start_date": "2013-12-27",
    "end_date": "2019-12-31",
    "timeseries": prices,
    "rebalance": True,
    "rebalance_frequency": "m"
}

In [159]:
mock_portfolio = Portfolio(**external_data)

In [160]:
projection = mock_portfolio.backtest_strategy()

In [161]:
sp500 = sp500.rename(columns={
    'Date':'date'
})

In [162]:
sp500.date = pd.to_datetime(sp500.date)

In [163]:
sp500 = sp500.sort_values(by='date').reset_index(drop=True)

In [164]:
sp500['market'] = sp500['Close/Last'] / sp500['Close/Last'].shift(periods=1) - 1

In [165]:
projection = projection.merge(sp500[['date','market']],how='left', on='date')

In [166]:
projection['portfolio'] = projection['portfolio'] / projection['portfolio'].shift(1) - 1

In [167]:
""" idx = pd.date_range(external_data['start_date'],external_data['end_date'],freq='D')
projection = projection.set_index(pd.DatetimeIndex(projection.date))
projection.reindex(idx, fill_value=0) """

" idx = pd.date_range(external_data['start_date'],external_data['end_date'],freq='D')\nprojection = projection.set_index(pd.DatetimeIndex(projection.date))\nprojection.reindex(idx, fill_value=0) "

In [178]:
Metrics().calculate_metrics(projection)

{'arithmetic_mean_m': -2.34037056084265,
 'arithmetic_mean_y': 32.62780868524565,
 'geometric_mean_m': nan,
 'geometric_mean_y': nan,
 'std_m': 11.582937501377577,
 'std_downside': 12.642413592390781,
 'cagr': nan,
 'sharpe_ratio': nan,
 'sortino_ratio': nan,
 'max_drawdown': -3.1078525977994698}

In [169]:
import statsmodels.formula.api as smf

In [171]:
projection.dropna()

,EBAY,AAPL,portfolio,date,market
1,29.905869,29.701655,-0.006541,2013-12-30,-0.000179
2,30.379291,30.049813,0.013783,2013-12-31,0.003960
3,29.705148,29.789625,-0.015462,2014-01-02,-0.008862
4,29.330667,29.135269,-0.017293,2014-01-03,-0.000333
5,28.515621,29.294145,-0.011223,2014-01-06,-0.002512
...,...,...,...,...,...
1508,78.763714,82.183368,0.000485,2019-12-24,-0.000195
1509,78.850722,83.813910,0.010672,2019-12-26,0.005128
1510,78.459187,83.782108,-0.002603,2019-12-27,0.000034
1511,77.871885,84.279366,-0.000555,2019-12-30,-0.005781


In [172]:
model = smf.ols(
    formula=f"portfolio ~ market", data=projection
)

results = model.fit()

In [177]:
pd.DataFrame(results.params)

,0
Intercept,0.000302
market,1.094092


In [153]:
projection

,EBAY,AAPL,portfolio,date,market,projection
0,30.000000,30.000000,0.006584,2013-12-27,-0.000337,0.006584
1,29.905869,29.701655,-0.013596,2013-12-30,-0.000179,-0.013596
2,30.379291,30.049813,0.015704,2013-12-31,0.003960,0.015704
3,29.705148,29.789625,0.017597,2014-01-02,-0.008862,0.017597
4,29.330667,29.135269,0.011351,2014-01-03,-0.000333,0.011351
...,...,...,...,...,...,...
1508,78.763714,82.183368,-0.010559,2019-12-24,-0.000195,-0.010559
1509,78.850722,83.813910,0.002609,2019-12-26,0.005128,0.002609
1510,78.459187,83.782108,0.000555,2019-12-27,0.000034,0.000555
1511,77.871885,84.279366,-0.007893,2019-12-30,-0.005781,-0.007893
